# Project Euler 104

The problem is to find the smallest Fibonacci number with the first 9 digits pandigital (containing 1..9 as digits), and also the last 9 digits pandigital.

In [1]:
from numba import jit
import numpy as np

In [2]:
@jit
def fibSimple():
    '''
    Simple Fibonacci.
    '''
    yield 0, 0
    n = 1
    prev = 0
    last = 1
    while True:
        yield n, last
        n = n + 1
        prev, last = last, prev+last

# This fails with @jit
def fibFloat():
    '''
    Infinite list of floating point numbers where the approximations to the Fibonacci numbers.
    '''
    # integers
    n = 0 # the nth Fibonacci number
    y = -1 # pull out a factor of 10**y
    # floats
    golden = (1 + np.sqrt(5)) / 2
    f = 10 / np.sqrt(5)
    
    while True:
        yield n, f, y
        n = n + 1
        f = golden * f
        if f > 10:
            f = f / 10
            y = y + 1
            

In [3]:
# Demonstrate this is really doing Fibonacci...
for ((ns, fs), (n, f, y)) in zip(fibSimple(), fibFloat()):
    if ns != n:
        raise ValueError('cannot even count')
    ff = np.round(f * 10**y)
    if fs != ff:
        raise ValueError('fail')
    print(n, fs, ff, f, y)
    if n > 20:
        break

0 0 0.0 4.472135955 -1
1 1 1.0 7.2360679775 -1
2 1 1.0 1.17082039325 0
3 2 2.0 1.894427191 0
4 3 3.0 3.06524758425 0
5 5 5.0 4.95967477525 0
6 8 8.0 8.0249223595 0
7 13 13.0 1.29845971347 1
8 21 21.0 2.10095194942 1
9 34 34.0 3.3994116629 1
10 55 55.0 5.50036361232 1
11 89 89.0 8.89977527522 1
12 144 144.0 1.44001388875 2
13 233 233.0 2.32999141628 2
14 377 377.0 3.77000530503 2
15 610 610.0 6.09999672131 2
16 987 987.0 9.87000202634 2
17 1597 1597.0 1.59699987477 3
18 2584 2584.0 2.5840000774 3
19 4181 4181.0 4.18099995216 3
20 6765 6765.0 6.76500002956 3
21 10946 10946.0 1.09459999817 4


In [4]:
@jit
def pandigitalFloat(f):
    '''
    Check if the first n digits of f are pandigital.
    Assumes f has one digit to the left of the decimal point,
    but does not check (for efficiency). After all, this has
    a specific use for which that is OK.
    '''
    buffer = np.zeros(10, dtype=bool)
    buffer[0] = True # if you find any zero, it cannot be 
    r = f
    for j in range(9):
        nf = np.floor(r)
        ni = int(nf)
        r = 10 * (r - nf) # expose one more digit
        if buffer[ni]:
            return False
        buffer[ni] = True
    return True # made it through 9 digits and found no 0, and no double counts

In [5]:
pandigitalFloat(1.2345678998765)

True

In [6]:
pandigitalFloat(9.98734566)

False

In [7]:
@jit
def startPanDigital():
    '''
    Infinite list of Fibonacci numbers that start off pandigital.
    '''
    for n, f, y in fibFloat():
        if y < 10:
            continue
        if pandigitalFloat(f):
            yield n, f, y
            

In [8]:
%%time
count = 12
for z in startPanDigital():
    print(z)
    count = count - 1
    if count <= 0:
        break

(2749, 1.4372689553389257, 574)
(4589, 4.9521763865736137, 958)
(7102, 7.5986432142729168, 1483)
(7727, 3.1478295605429634, 1614)
(8198, 8.5347296185526567, 1712)
(9383, 3.8154297647901476, 1960)
(12633, 6.1854923741889349, 2639)
(15708, 2.6814397584614751, 3282)
(19014, 2.1953648727152033, 3973)
(21206, 2.769581431772437, 4431)
(21303, 5.1786342956357068, 4451)
(21434, 1.234798564357511, 4479)
CPU times: user 547 ms, sys: 3.39 ms, total: 550 ms
Wall time: 549 ms


This confirms that the first leading pandigital Fibonacci number is $F_{2749}$, which has 575 digits. Also, there are others in the list, and they are fairly efficient to find.

So now that I have a generator that lists the (probably infinite) list of Fibonacci's that are leading pandigital, I need to filter out the ones that are trailing pandigital. For this, I use the fact that
$$
\left(\begin{array}{cc} F_{n+1} & F_n \\ F_n & F_{n-1} \end{array}\right)
= \left(\begin{array}{cc} 1 & 1 \\ 1 & 0 \end{array}\right)^n \mod m
$$
(see the Wikipedia page on Fibonacci numbers). 
Thus, if $m = 10^9$, we may easily compute the last $9$ digits of $F_n$ for any large $n$, and check
it for theh pandigital property.

In [9]:
@jit
def matN_modM(A, n, m):
    '''
    Compute the matrix A**n mod m
    This method is proportional to log_2 n, so can work with large n.
    '''
    if n == 0:
        return np.eye(A.shape[0], dtype=int)
    if np.mod(n, 2) == 0:
        h = n // 2 # half of n as an integer
        M = matN_modM(A, h, m)
        return np.mod(M * M, m)
    else:
        M = matN_modM(A, n-1, m)
        return np.mod(A * M, m)
        
@jit    
def fibModular(n, m=1000000000):
    '''
    Compute F_n mod m as an integer.
    '''
    A = np.matrix([[1,1],[1,0]], dtype=int)
    An = matN_modM(A, n, m)
    return An[0,1]

In [10]:
A = np.matrix([[1,1],[1,0]], dtype=int)
A ** 20

matrix([[10946,  6765],
        [ 6765,  4181]])

In [11]:
matN_modM(A, 20, 1000000)

matrix([[10946,  6765],
        [ 6765,  4181]])

In [12]:
fibModular(20)

6765

Thus, the thing works for moderate sized Fibonacci numbers. 

One more check from the Project Euler problem statement verifies that $F_{541}$ is ending pandigital:

In [13]:
fibModular(541)

839725641

In general, I need a pandigital checker for an integer now...

In [14]:
@jit
def pandigitalInt(n):
    '''
    Decide whether or not the integer n is pandigital.
    '''
    buffer = np.zeros(10, dtype=bool)
    buffer[0] = True # if you find any zero, it cannot be 
    for j in range(9):
        d, n = np.mod(n, 10), n // 10
        if buffer[d]:
            return False
        buffer[d] = True
    if n == 0:
        return True # made it through 9 digits and found no 0, and no double counts
    else:
        return False

In [15]:
pandigitalInt(123456789)

True

In [16]:
pandigitalInt(120456789)

False

In [17]:
pandigitalInt(fibModular(541))

True

Thus, I can put together the previous generator for leading pandigital $F$ with a filter for following pandigital numbers...

In [18]:
@jit
def bothPandigital():
    '''
    Generate an infinite list of n for which $F_n$ is leading
    and following pandigital.
    '''
    for n, f, y in startPanDigital():
        fnMod = fibModular(n)
        if pandigitalInt(fnMod):
            yield n, fnMod, f, y+1

In [19]:
%%time
for z in bothPandigital():
    print(z)
    break

(329468, 352786941, 2.4568173918884835, 68855)
CPU times: user 5.84 s, sys: 5.19 ms, total: 5.84 s
Wall time: 5.84 s


Therefore, the first one is $F_{329468}$, a $68855$ digit number that has leading digits $245681739$, and trailing digits $352786941$.

It takes between 5 and 6 seconds to compute on my Mac.

It takes somewhat longer to compute more of them...

In [20]:
%%time
count = 2
for z in bothPandigital():
    print(z)
    count = count - 1
    if count <= 0:
        break

(329468, 352786941, 2.4568173918884835, 68855)
(6496145, 197283645, 4.6215397847121347, 1357614)
CPU times: user 1min 51s, sys: 48.4 ms, total: 1min 51s
Wall time: 1min 51s


So, I found the second one, $F_{6496145}$, a $1357614$ digit number that has leading digits $462153978$, and trailing digits $197283645$, in a little less than two minutes.